In [1]:
# Define initial prompt and output format
initial_prompt = (
    "Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: // suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3. explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave blank for 'risk not present.'"
)

output_format_prompt = (
    "Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'"
)

In [2]:
# Define output schema
output_schema = {
    'key_to_extract': 'risk_output',
    'value_mapping': {
        'risk_present': 1,
        'risk_not_present': 0
    },
    'regex_pattern': r'"risk_output":\s*"(.*?)"'
}

In [3]:
# Set number of optimization iterations
iterations = 3

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "risks.csv"

------------------------------------------------------------------------------------------

In [6]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [7]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['text', 'label'])
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=4, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (8, 2)
                                                text  label
0  Please confirm if the final balance on the acc...      0
1  Consumer said they are trying to avoid harming...      1
2  Good afternoon <person>, I refer to the compla...      0
3            Consumer had suicide attempt in January      1
4  Could you provide a gesture of goodwill for th...      0


In [8]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 17760
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention    │
│ suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do     │
│ they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: //           │
│ suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case  │
│ 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case         │
│ contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3.  │
│ explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your         │
│ explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave      │
│ blank for 'risk not present.'                                                                                   │
│                                                                                                                 │
│ Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and          │
│ explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processing text 1/8
Unable to find risk_output in the raw output
Prediction 1/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: This conversation just started. We haven't discussed any previous topics or accounts yet, so I can't provide an update on a specific balance or identify any risks. 

However, based on our conversation earlier, we were discussing identifying risk factors for client self-harm. Would you like to continue that topic?
Processing text 2/8
Prediction 2/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 3/8
Prediction 3/8: 0 | Ground Truth: 0 ✅ (TN)
Processing text 4/8
Prediction 4/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 5/8
Unable to find risk_output in the raw output
Prediction 5/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: I'd be happy to provide some text that could be used as a token of goodwill.

Here are a few options:

* A free consultation or analysis on a future project
* A voucher for a discount on a future service
* 

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 1.0000 │
│ Recall              │ 0.7500 │
│ Accuracy            │ 0.8000 │
│ F1-score            │ 0.8571 │
│ Valid Predictions   │      5 │
│ Invalid Predictions │      3 │
└─────────────────────┴────────┘


Analyzing misclassifications...


...


Analyzing misclassifications...


╭──────────────────────────────────────── Analysis of Misclassifications ─────────────────────────────────────────╮
│                                                                                                                 │
│ **Analysis of Misclassifications**                                                                              │
│                                                                                                                 │
│ After reviewing the provided texts and analyzing the misclassifications, I identified several instances where   │
│ the LLM model missed subtle indicators of positive cases.                                                       │
│                                                                                                                 │
│ **False Negatives (0) Texts Incorrectly Classified as Positive:**                                               │
│                                                                                                                 │
│ These texts were actually correctly classified as negative by the LMM. However, upon closer examination, it     │
│ appears that these texts do not contain any explicit or implicit references to self-harm risk. The language     │
│ used is generally innocuous and does not suggest a risk of self-harm.                                           │
│                                                                                                                 │
│ **False Positives (1) Texts Incorrectly Classified as Negative:**                                               │
│                                                                                                                 │
│ The following texts were incorrectly classified as negative by the LMM:                                         │
│                                                                                                                 │
│ Text 1:                                                                                                         │
│ "Client mentions feeling overwhelmed due to a heavy workload, but no mention of suicidal ideation or specific   │
│ plans for self-harm. However, they do express a desire to 'escape' their situation."                            │
│                                                                                                                 │
│ Text 2:                                                                                                         │
│ "Client has a history of suicidal thoughts and ideation in the past, but currently reports being on medication  │
│ and attending therapy sessions regularly. They also express feelings of hopelessness and despair about their    │
│ current situation, stating it's 'not worth living.'"                                                            │
│                                                                                                                 │
│ **Analysis of Misclassifications:**                                                                             │
│                                                                                                                 │
│ Upon reviewing these misclassifications, I identified several patterns where the LLM model may have missed      │
│ subtle indicators of positive cases:                                                                            │
│                                                                                                                 │
│ 1.  **Ambiguous Language:** In Text 2, the client expresses a desire to "escape" their situation, which could   │
│ be interpreted as a sign of self-harm risk.                                                                     │
│ 2.  **Subtle Indicators:** The client in Text 2 mentions having a history of suicidal thoughts and ideation,    │
│ but currently reports being on medication and attendin


Generating new prompt...

Iteration 3/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ I can improve the provided prompt to increase recall while maintaining precision.                               │
│                                                                                                                 │
│ Revised Prompt:                                                                                                 │
│                                                                                                                 │
│ **Please assess the client's self-harm risk accurately**                                                        │
│                                                                                                                 │
│ When evaluating the following risk factor: Is the client at risk of self-harm? Please consider both current and │
│ past experiences. For instance, do they mention suicidal thoughts or ideation? Do they imply they might do      │
│ physical damage to themselves or to property? Do they reference wanting to 'end it all' or say it's 'not worth  │
│ living'? Take into account the client's past struggles and ongoing mental health issues when making your        │
│ assessment.                                                                                                     │
│                                                                                                                 │
│ Be cautious of subtle indicators, such as ambiguous language or indirect expressions that may suggest a risk of │
│ self-harm. Consider contextual information like the client's overall mental health status and any current       │
│ safeguarding issues when evaluating their situation.                                                            │
│                                                                                                                 │
│ If you identify evidence of self-harm risk, please provide specific direct quotes or phrases from the text to   │
│ support your conclusion. When making your assessment, prioritize caution over uncertainty, and prefer           │
│ outputting 'risk present' if there are any indications of potential harm.                                       │
│                                                                                                                 │
│ Output should be STRICT JSON, containing:                                                                       │
│                                                                                                                 │
│ *   `'risk_type'`: If applicable, indicate the type of risk ('suicidality').                                    │
│ *   `'risk_output'`: Provide a binary response indicating whether you've identified evidence of self-harm risk  │
│ or not. Use 'risk present' if you conclude there's an indication of self-harm risk, and 'risk not present'      │
│ otherwise.                                                                                                      │
│ *   `'explanation'`: If the output is 'risk present', please provide specific direct quotes or phrases from the │
│ text that support your conclusion. Be concise but accurate in your explanation.                                 │
│                                                                                                                 │
│ Examples for `risk_output`:                                                                                     │
│                                                                                                                 │
│     *   {'risk_type': 'suicidality', 'risk_output': 'risk present', 'explanation': ['Client expressed suicidal  │
│ ideation']}.                                                                                                    │
│     *   {'risk_type': '', 'risk_output': 'risk not pre

Processing text 1/8
Unable to find risk_output in the raw output
Prediction 1/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: This conversation just started. We haven't discussed anything yet, including accounts or balances. If you'd like to discuss something specific, I'm here to help!
Processing text 2/8
Prediction 2/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 3/8
Unable to find risk_output in the raw output
Prediction 3/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: Good afternoon, I appreciate your follow-up. The meter exchange for Mr. [person] was indeed carried out on the specified date of [date]. Everything went smoothly without any complications. As a result, the issue with his electricity supply has been resolved.

Regarding further updates on the case, our records indicate that all necessary corrections have been made to his account. Our team is keeping an eye on things, but so far, there haven't been any additional issues reported by 